# Comparação de Modelos

## Previsões

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('./dados/dados.csv') #Lendo os dados

In [2]:
#Declarando as variáveis:
X = df[['Radiacao (KJ/m²)', 'Temp. Ins. (C)', 'Umi. Ins. (%)', 'Chuva (mm)', 'Hora', 'Mês', 'Pto Orvalho Ins. (C)', 'Temp. Max. (C)',
       'Temp. Min. (C)']]

y = df['Potência CA (W)']


#Separando em treino e teste:
X_t, X_test, y_t, y_test = train_test_split(X, y, test_size=0.1, random_state=101) #10% dos dados são p/ teste

### SVR

In [3]:
from sklearn.svm import SVR
reg = SVR(C=40, epsilon=0.1)

#Normalizando os dados de entrada:
scaler = StandardScaler()
X_t_scaled = scaler.fit_transform(X_t)
X_test_scaled = scaler.fit_transform(X_test)

y_t_log = np.log(y_t) #Aplicando transformação log nos dados de saída

#Treinando o modelo:
reg.fit(X_t_scaled, y_t_log)

#Fazendo a previsão:
pred_log = reg.predict(X_test_scaled)
svr = np.exp(pred_log)

### Árvore de Decisão

In [4]:
from sklearn.tree import DecisionTreeRegressor
reg = DecisionTreeRegressor(min_samples_leaf=16)

reg.fit(X_t, y_t)

dt = reg.predict(X_test)

### KNR

In [5]:
from sklearn.neighbors import KNeighborsRegressor
reg = KNeighborsRegressor(n_neighbors=9,weights='distance', p=2)

#Normalizando os dados de entrada:
scaler = StandardScaler()
X_t_scaled = scaler.fit_transform(X_t)
X_test_scaled = scaler.fit_transform(X_test)


#Treinando o modelo:
reg.fit(X_t_scaled, y_t)

#Fazendo a previsão:
knr = reg.predict(X_test_scaled)

### RNA

In [6]:
from sklearn.neural_network import MLPRegressor
layers = (50,50,50,50,50)
reg = MLPRegressor(hidden_layer_sizes = layers, activation='relu', solver='lbfgs', learning_rate='adaptive', random_state=101)

#Normalizando os dados de entrada:
scaler = StandardScaler()
X_t_scaled = scaler.fit_transform(X_t)
X_test_scaled = scaler.fit_transform(X_test)


#Treinando o modelo:
reg.fit(X_t_scaled, y_t)

#Fazendo a previsão:
rna = reg.predict(X_test_scaled)

## Teste de Dunnett

In [16]:
a=5
b=len(y_test)

tot_y_test = sum(y_test)
tot_svr = sum(svr)
tot_dt = sum(dt)
tot_knr = sum(knr)
tot_rna = sum(rna)

soma = tot_y_test + tot_svr + tot_dt + tot_knr + tot_rna
somaQ = sum(y_test**2) + sum(svr**2) + sum(dt**2) + sum(knr**2) + sum(rna**2)

SQtotal = somaQ-((soma**2)/(a*b))

SQtrat = ((tot_y_test**2 + tot_svr**2 + tot_dt**2 + tot_knr**2 + tot_rna**2)/b) - (soma**2/(a*b))

SQres = SQtotal - SQtrat

QM = SQres/(a*(b-1))

648228.9142007054


In [17]:
DMS = 2.442*np.sqrt((2*QM)/(b))

print(DMS)

112.95060064975534


In [23]:
print('Support Vector Regression: ', np.mean(y_test) - np.mean(svr))

print('Árvore de Decisão: ', np.mean(y_test) - np.mean(dt))

print('KNeighbors Regressor: ', np.mean(y_test) - np.mean(knr))

print('Rede Neural: ', np.mean(y_test) - np.mean(rna))

Support Vector Regression:  1.9624248529853503
Árvore de Decisão:  3.5161408602141364
KNeighbors Regressor:  28.718521446338627
Rede Neural:  13.244452221988695


Todos as diferenças são menores do que o DMS, portanto pode-se dizer que não há diferença significativa entre as médias.